Load dataset

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Import libraries

In [2]:
import os
from glob import glob
from pathlib import Path
from collections import defaultdict
from pprint import pprint
from random import choice, sample
import cv2

Check GPU availability

Install tensorflow-gpu

In [3]:
!pip3 install tensorflow-gpu==1.13.1

  Using cached tensorflow_estimator-1.13.0-py2.py3-none-any.whl (367 kB)
  Using cached tensorboard-1.13.1-py3-none-any.whl (3.2 MB)
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.7.0
    Uninstalling tensorflow-estimator-2.7.0:
      Successfully uninstalled tensorflow-estimator-2.7.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.7.0 requires tensorboard~=2.6, but you have tensorboard 1.13.1 which is incompatible.
tensorflow 2.7.0 requires tensorflow-estimator<2.8,~=2.7.0rc0, but you have tensorflow-estimator 1.13.0 which is incompatible.
tensorflow-text 2.8.1 requires tensorflow<2.9,>=2.8.0, but you have tensorflow 2.7.0 w

Set up object-detection api

In [4]:
!git clone https://github.com/tensorflow/models.git

fatal: destination path 'models' already exists and is not an empty directory.


In [5]:
!ls

cocoapi  drive	models	sample_data


Set up cocoapi

In [6]:
!pip install cython
!git clone https://github.com/cocodataset/cocoapi.git
%cd ./cocoapi/PythonAPI
!ls
!make
!cp -r pycocotools ../../models/research/
%cd ../../models/research
!ls | grep pycocotools

fatal: destination path 'cocoapi' already exists and is not an empty directory.
/content/cocoapi/PythonAPI
Makefile  pycocoDemo.ipynb  pycocoEvalDemo.ipynb  pycocotools  setup.py
python setup.py build_ext --inplace
running build_ext
skipping 'pycocotools/_mask.c' Cython extension (up-to-date)
building 'pycocotools._mask' extension
creating build
creating build/common
creating build/temp.linux-x86_64-3.7
creating build/temp.linux-x86_64-3.7/pycocotools
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fdebug-prefix-map=/build/python3.7-pX47U3/python3.7-3.7.12=. -fstack-protector-strong -Wformat -Werror=format-security -g -fdebug-prefix-map=/build/python3.7-pX47U3/python3.7-3.7.12=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/local/lib/python3.7/dist-packages/numpy/core/include -I../common -I/usr/include/python3.7m -c ../common/maskApi.c -o build/temp.linux-x86_64-3.7/../common/mask

Compile protobuf

In [7]:
!pwd
!protoc object_detection/protos/*.proto --python_out=.

/content/models/research


Install object_detection api

In [8]:
%cd /content/models/research
!pwd
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .

/content/models/research
/content/models/research
Processing /content/models/research
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 497.5 MB 24 kB/s 
  Using cached tensorboard-2.8.0-py3-none-any.whl (5.8 MB)
     |████████████████████████████████| 1.4 MB 38.4 MB/s 
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1686356 sha256=41af81769e659b9b16106006666068af2cd7eba6c98fc3c233cf6cc386a2ff3e
  Stored in directory: /tmp/pip-ephem-wheel-cache-z8w_wror/wheels/fa/a4/d2/e9a5057e414fd46c8e543d2706cd836d64e1fcd9eccceb2329
Successfully built object-detection
  Attemptin

Test to see if object detection api is successfully installed

In [9]:
# should only executed once
!pip install tensorflow==2.7
#!tf-models-official==2.7.0
#!tensorflow_io==0.23.1
!pip install opencv-python-headless==4.1.2.30

  Using cached tensorflow-2.7.0-cp37-cp37m-manylinux2010_x86_64.whl (489.6 MB)
  Using cached keras-2.7.0-py2.py3-none-any.whl (1.3 MB)
  Using cached tensorflow_estimator-2.7.0-py2.py3-none-any.whl (463 kB)
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 1.13.0
    Uninstalling tensorflow-estimator-1.13.0:
      Successfully uninstalled tensorflow-estimator-1.13.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.8.0
    Uninstalling tensorflow-2.8.0:
      Successfully uninstalled tensorflow-2.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-models-official 2.8.0 requires tensorflow~=2.8.0, but you have tensorflow 2.7.0 which i

In [10]:
!pwd
!pip list | grep tensor
%cd /content/models/research
!python object_detection/builders/model_builder_tf2_test.py

/content/models/research
tensorboard                   2.8.0
tensorboard-data-server       0.6.1
tensorboard-plugin-wit        1.8.1
tensorflow                    2.7.0
tensorflow-addons             0.16.1
tensorflow-datasets           4.0.1
tensorflow-estimator          2.7.0
tensorflow-gcs-config         2.8.0
tensorflow-gpu                1.13.1
tensorflow-hub                0.12.0
tensorflow-io                 0.24.0
tensorflow-io-gcs-filesystem  0.24.0
tensorflow-metadata           1.7.0
tensorflow-model-optimization 0.7.2
tensorflow-probability        0.16.0
tensorflow-text               2.8.1
/content/models/research
Running tests under Python 3.7.12: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
2022-03-19 22:38:52.682167: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
W0319 22:38:53.692625 139863531890

Download the models and retrain them!

In [11]:
%cd /content
!ls drive/MyDrive/COMS4995_Competition/object_detection_workspace/models
# cd ../..

/content
centernet_hg104_512x512_coco17_tpu-8
efficientdet_d4_coco17_tpu-32
faster_rcnn_inception_resnet_v2_640x640_coco17_tpu-8


Fine-tuning the models

In [12]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()
print("Found GPU at: {}".format(device_name))

!nvidia-smi

Found GPU at: /device:GPU:0
Sat Mar 19 22:39:36 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    63W / 149W |    160MiB / 11441MiB |      2%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                            

In [13]:
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libcudnn8 is already the newest version (8.1.0.77-1+cuda11.2).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [14]:
!pip list | grep tensor
%cd /content/drive/MyDrive/COMS4995_Competition/object_detection_workspace
!python model_main_tf2.py --pipeline_config_path=models/centernet_hg104_512x512_coco17_tpu-8/v1/pipeline.config --model_dir=models/centernet_hg104_512x512_coco17_tpu-8/v1/ --checkpoint_every_n=100 --num_workers=1 alsologtostderr


tensorboard                   2.8.0
tensorboard-data-server       0.6.1
tensorboard-plugin-wit        1.8.1
tensorflow                    2.7.0
tensorflow-addons             0.16.1
tensorflow-datasets           4.0.1
tensorflow-estimator          2.7.0
tensorflow-gcs-config         2.8.0
tensorflow-gpu                1.13.1
tensorflow-hub                0.12.0
tensorflow-io                 0.24.0
tensorflow-io-gcs-filesystem  0.24.0
tensorflow-metadata           1.7.0
tensorflow-model-optimization 0.7.2
tensorflow-probability        0.16.0
tensorflow-text               2.8.1
/content/drive/MyDrive/COMS4995_Competition/object_detection_workspace
2022-03-19 22:39:52.910381: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
I0319 22:39:52.916572 13997

Use retrained models to generate labels

In [15]:
import os
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import glob


PATH_TO_MODEL_DIR = './drive/MyDrive/COMS4995_Competition/object_detection_workspace/models/faster_rcnn_inception_resnet_v2_640x640_coco17_tpu-8/v1'
PATH_TO_CFG = PATH_TO_MODEL_DIR + '/pipeline.config'
PATH_TO_CKPT = PATH_TO_MODEL_DIR + '/checkpoints'
PATH_TO_LABELS = '../workspace/data/label_map.pbtxt'

PATH_TO_DATASET_DIR = '../BPP_train'
PATH_TO_VALIDATION_DIR = PATH_TO_DATASET_DIR + '/haze_test'
PATH_TO_TRAIN_DIR = PATH_TO_DATASET_DIR + '/haze_train'
PATH_TO_CLEAN_VALIDATION_DIR = PATH_TO_DATASET_DIR + '/dehaze_test'
PATH_TO_CLEAN_TRAIN_DIR = PATH_TO_DATASET_DIR + '/dehaze_train'
PATH_TO_RESULT_LABEL_DIR = './results'
PATH_TO_DRYRUN_DIR = '../train/dry-run-1'


validation_images = glob.glob(PATH_TO_VALIDATION_DIR + '/*.jpg')
train_images = glob.glob(PATH_TO_TRAIN_DIR + '/*.jpg')
clean_validation_images = glob.glob(PATH_TO_CLEAN_VALIDATION_DIR + '/*.jpg')
clean_train_images = glob.glob(PATH_TO_CLEAN_TRAIN_DIR + '/*.jpg')
dry_run_images = glob.glob(PATH_TO_DRYRUN_DIR + '/*.jpg')
dehaze_dry_run_images = glob.glob('./AOD-Net/data/result/*.jpg')


# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(PATH_TO_CFG)
model_config = configs['model']
print(model_config)
detection_model = model_builder.build(model_config=model_config, is_training=False)
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(PATH_TO_CKPT, 'ckpt-545')).expect_partial()
im_width = 1845
im_height = 750
from pathlib import Path


@tf.function
def detect_fn(image):
    """Detect objects in image."""

    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    # print(detections)

    return detections

'''
test_im_path = train_images[0]
hazy_im, _ = get_hazy_clean_image(test_im_path)
tensor_im = tf.convert_to_tensor(hazy_im)
converted_img = tf.image.convert_image_dtype(tensor_im, tf.float32)[tf.newaxis, ...]
detections = detect_fn(converted_img)
# detections = detection_model(converted_img)
result = {key: value.numpy() for key, value in detections.items()}
'''

def filter_coords(result, threshold=0.0):
    '''
      A helper method which extracts the bounding boxes and threshold confidence scores
      from the data structure returned from the pretrained model

      Parameters:
              result : Output of the pretrained tensor flow model
              threshold : Optional threshold confidence above which bounding boxes are selected
      Returns:
              The list of coordinates of the detected objects and the threshold score
    '''
    class_entities = result['detection_classes']
    detection_scores = result['detection_scores']
    bounding_boxes = result['detection_boxes']
    detection_boxes_with_thresholds = []
    for detection_class, detection_score, bounding_box in zip(class_entities, detection_scores, bounding_boxes):
        if 1 == detection_class and detection_score >= threshold:
            detection_boxes_with_thresholds.append((bounding_box, detection_score))
    return detection_boxes_with_thresholds


# f = filter_coords(result, threshold=0.5)
'''
coords = result['detection_boxes']
width = 750
height = 1845
converted_coords = []
print(coords)
for coord in coords[0]:
    print(coord)
    ymin, xmin, ymax, xmax = coord
    converted_coord = [int(xmin * width), int(ymin * height), int(xmax * width), int(ymax * height)]
    converted_coords.append(converted_coord)

print(converted_coords)
visualize_bb_image(val_images[0], converted_coords)
'''


def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
      path: the file path to the image

    Returns:
      uint8 numpy array with shape (img_height, img_width, 3)
    """
    return np.array(Image.open(path))


for image_path in dehaze_dry_run_images:

    print('Running inference for {}... '.format(image_path), end='')

    image_np = load_image_into_numpy_array(image_path)

    # Things to try:
    # Flip horizontally
    # image_np = np.fliplr(image_np).copy()

    # Convert image to grayscale
    # image_np = np.tile(
    #     np.mean(image_np, 2, keepdims=True), (1, 1, 3)).astype(np.uint8)

    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)

    detections = detect_fn(input_tensor)

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    label_id_offset = 1
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64) + label_id_offset

    image_np_with_detections = image_np.copy()

    # visualize the detected box on the image
    viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes'],
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=10,
            min_score_thresh=0.05,
            agnostic_mode=False)

    plt.figure()
    filename = '{}.jpg'.format(Path(image_path).stem)
    plt.imshow(image_np_with_detections)
    plt.savefig('results_fig/'+filename)

    # output the detected box to the file
    coords = filter_coords(detections, threshold=0.05)
    newfile = os.path.join(PATH_TO_RESULT_LABEL_DIR, Path(image_path).stem + '.txt')
    with open(newfile, "w+") as f:
        for coord in coords:
            ymin, xmin, ymax, xmax = coord[0]
            threshold = coord[1]
            format_string = f'vehicle {xmin * im_width} {ymin * im_height} {xmax * im_width} {ymax * im_height} {threshold}\n'
            f.write(format_string)
    print('Done')

NotFoundError: ignored